In [42]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import numpy as np
m=5
mesh = StructuredTesseraktMesh(m,m,m,m)

In [43]:
from methodsnm.fes import *
fes = P1_Tesserakt_Space(mesh)

## Verification Example for the timeintegral: Solving the ODE *uₜ = t²*

In the following, we consider the simple ordinary differential equation:

$
\frac{\partial u}{\partial t} = t^2
$

This problem is purely time-dependent and serves as a minimal test case to verify the correct implementation of the time integral assembling procedure in the finite element framework.

# Be Careful!!!
Be careful: if you choose m too small, you will end up with very few interior DOFs.
The matrix entries

$$ B_{ij} = \int \varphi_i\,\partial_t \varphi_j $$

vanish for all i ≠ j, and even the diagonal term

$$ \int (\partial_t \varphi_i)\,\varphi_i $$

is zero. As a result, the entire B-matrix collapses to nothing, which produces very poor results.
Try m = 2 to see the effect — you will have only one interior DOF.


In [44]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin

blf = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf += TimeIntegral(c)
blf.assemble()

lf = LinearForm(fes)
f = GlobalFunction(lambda x: 3*x[3]**2, mesh = mesh)
lf += SourceIntegral(f)
lf.assemble()

In [45]:
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

uh = FEFunction(fes)
uexact = lambda x: x[3]**3
uh._set(uexact, bndry = mesh.bndry_vertices)
res = lf.vector - blf.matrix.dot(uh.vector)

from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)
from methodsnm.forms import compute_difference_L2

uex = FEFunction(fes)
uex._set(uexact)
fex = GlobalFunction(uexact,mesh)
l2diff = compute_difference_L2(fex, uex, mesh, intorder = 5)
print("l2diff =", l2diff)



[0. 0. 0. ... 1. 1. 1.]
l2diff = 0.012588732679511066


## Verification Example for the Laplacian without `TimeIntegral`:
### Solving the PDE $$\Delta_x u = f(x)$$

In the following, we consider the simple partial differential equation

$$
-\Delta u(x) = 3 \pi \cos(\pi x_0)\,\cos(\pi x_1)\,\cos(\pi x_2), \quad x \in \Omega,
$$

with the domain given by the space-time cylinder $\Omega \times T$.
On the spatial boundary we impose the Dirichlet condition

$$
u(x) = 3 \pi \cos(\pi x_0)\,\cos(\pi x_1)\,\cos(\pi x_2), x \in \delta \Omega
$$

This problem is purely spatial (no dependence on the time coordinate) and serves as a minimal test case to verify that assembling without a `TimeIntegral` recovers the correct spatial Laplacian structure inside the space-time finite element formulation.

### Analytical Solution

The exact solution to the PDE

$$
-\Delta u(x) = 3 \pi \cos(\pi x_0)\,\cos(\pi x_1)\,\cos(\pi x_2)
$$
is
$$
u(x) = \cos(\pi x_0)\,\cos(\pi x_1)\,\cos(\pi x_2),
$$




In [46]:
blf3 = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf3 += LaplaceIntegral_without_time(c)
blf3.assemble()

lf3 = LinearForm(fes)
f3 = GlobalFunction(lambda x: 3*pi*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2]), mesh = mesh)
lf3 += SourceIntegral(f3)
lf3.assemble()

def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

uh3 = FEFunction(fes)
f4 = lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])
uh3._set(f4, True)
res = lf3.vector - blf3.matrix.dot(uh3.vector)

from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
uh3.vector += solve_on_freedofs(blf3.matrix,res,freedofs)
from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(f4, mesh = mesh)
l2diff = compute_difference_L2(uh3, uex, mesh, intorder = 5)
print(l2diff)

0.04703633016895289


## Verification Example: Solving the parabolic PDE 

In the following, we consider the simple ordinary differential equation:

$-\Delta u + \frac{\partial u}{\partial t} = f$

This problem is the mix off the two problems before and serves as a minimal test case to verify the correct implementation of the time integral + laplaceintegral assembling procedure in the finite element framework.

The right-hand side is

$$
f(x) = 0,
$$

and

$$
u(x) = \sin(\pi x_0)\,\sin(\pi x_1)\,\sin(\pi x_2)
$$

is prescribed on the spatial boundary.  
The analytical solution of the space-time problem is

$$
u(x) = \sin(\pi x_0)\,\sin(\pi x_1)\,\sin(\pi x_2)\,e^{-3\pi^2 t}.
$$


In [ ]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin

blf3 = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf3 += TimeIntegral(c)
blf3 += LaplaceIntegral_without_time(c)
blf3.assemble()

lf3 = LinearForm(fes)
f3 = GlobalFunction(lambda x: 0, mesh = mesh)
lf3 += SourceIntegral(f3)
lf3.assemble()

def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

top = mesh.filter_bndry_points("max",3)
bndry = list_diff(mesh.bndry_vertices,top)

uh = FEFunction(fes)
f = lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2])
uh._set(f, bndry)
res = lf3.vector - blf3.matrix.dot(uh.vector)

from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,bndry)
uh.vector += solve_on_freedofs(blf3.matrix,res,freedofs)

from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2])*np.exp(-3*pi**2*x[3]), mesh = mesh)
l2diff = compute_difference_L2(uh, uex, mesh, intorder = 5)

print("l2diff =", l2diff)

l2diff = 0.08823185614692797
